<a href="https://colab.research.google.com/github/Ajax121/iNeuron/blob/master/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

In [15]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

In [16]:
df['FlightNumber'] = df.FlightNumber.interpolate().astype(int)
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   From_To       5 non-null      object
 1   FlightNumber  5 non-null      int64 
 2   RecentDelays  5 non-null      object
 3   Airline       5 non-null      object
dtypes: int64(1), object(3)
memory usage: 288.0+ bytes


2. The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

In [18]:
temp_df = df['From_To'].str.split('_', expand=True)
temp_df.columns = ['From','To']
temp_df

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)

In [19]:
temp_df['From'] = temp_df['From'].str.title()
temp_df['To'] = temp_df['To'].str.title()

temp_df

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

In [20]:
df = df.drop('From_To',axis=1)

In [21]:
df = temp_df.join(df)
df

,From,To,FlightNumber,RecentDelays,Airline
0,London,Paris,10045,"[23, 47]",KLM(!)
1,Madrid,Milan,10055,[],<Air France> (12)
2,London,Stockholm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest,Paris,10075,[13],12. Air France
4,Brussels,London,10085,"[67, 32]","""Swiss Air"""


5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [22]:
delays = pd.DataFrame(df['RecentDelays'].tolist(),index=df.index)
delays

,0,1,2
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [23]:
delays.columns = ['delay_1','delay_2','delay_3']
delays

,delay_1,delay_2,delay_3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [24]:
df = df.drop('RecentDelays',axis=1)
temp = df.pop('Airline')

In [25]:
df = df.join(delays).join(temp)
df

,From,To,FlightNumber,delay_1,delay_2,delay_3,Airline
0,London,Paris,10045,23.0,47.0,NaN,KLM(!)
1,Madrid,Milan,10055,NaN,NaN,NaN,<Air France> (12)
2,London,Stockholm,10065,24.0,43.0,87.0,(British Airways. )
3,Budapest,Paris,10075,13.0,NaN,NaN,12. Air France
4,Brussels,London,10085,67.0,32.0,NaN,"""Swiss Air"""


In [26]:
df['Airline'] = df['Airline'].str.replace('[^a-zA-Z]', ' ')
df

,From,To,FlightNumber,delay_1,delay_2,delay_3,Airline
0,London,Paris,10045,23.0,47.0,NaN,KLM
1,Madrid,Milan,10055,NaN,NaN,NaN,Air France
2,London,Stockholm,10065,24.0,43.0,87.0,British Airways
3,Budapest,Paris,10075,13.0,NaN,NaN,Air France
4,Brussels,London,10085,67.0,32.0,NaN,Swiss Air
